# [**LCA net**](https://arxiv.org/pdf/2008.10325v1.pdf) 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import random
from PIL import Image
import time
import datetime

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing and loading of data

In [ ]:
#ls drive/MyDrive/reside/archive/clear_images drive/MyDrive/reside/archive/haze  

In [58]:
# function to load the image in the form of tensors.

def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = (412,548), antialias = True)
    img = img / 255.0
    return img

In [4]:
# function to get the path of individual image.

def data_path(orig_img_path, hazy_img_path):
    
    train_img = []
    val_img = []
    
    orig_img = glob.glob(orig_img_path + '/*.jpg')
    n = len(orig_img)
    random.shuffle(orig_img)
    train_keys = orig_img[:int(0.9*n)]        #90% data for train, 10% for test
    val_keys = orig_img[int(0.9*n):]
    
    split_dict = {}
    for key in train_keys:
        split_dict[key] = 'train'
    for key in val_keys:
        split_dict[key] = 'val'
        
    hazy_img = glob.glob(hazy_img_path + '/*.jpg')
    for img in hazy_img:
        img_name = img.split('/')[-1]
        orig_path = orig_img_path + '/' + img_name.split('_')[0] + '.jpg'
        if (split_dict[orig_path] == 'train'):
            train_img.append([img, orig_path])
        else:
            val_img.append([img, orig_path])
            
    return train_img, val_img

In [5]:
# function to load tensor image data in batches.

def dataloader(train_data, val_data, batch_size):
    
    train_data_orig = tf.data.Dataset.from_tensor_slices([img[1] for img in train_data]).map(lambda x: load_image(x))
    train_data_haze = tf.data.Dataset.from_tensor_slices([img[0] for img in train_data]).map(lambda x: load_image(x))
    train = tf.data.Dataset.zip((train_data_haze, train_data_orig)).shuffle(buffer_size=100).batch(batch_size)
    
    val_data_orig = tf.data.Dataset.from_tensor_slices([img[1] for img in val_data]).map(lambda x: load_image(x))
    val_data_haze = tf.data.Dataset.from_tensor_slices([img[0] for img in val_data]).map(lambda x: load_image(x))
    val = tf.data.Dataset.zip((val_data_haze, val_data_orig)).shuffle(buffer_size=100).batch(batch_size)
    
    return train, val

In [6]:
# function to display output.

def display_img(model, hazy_img, orig_img):
    
    dehazed_img = model(hazy_img, training = True)
    plt.figure(figsize = (15,15))
    
    display_list = [hazy_img[0], orig_img[0], dehazed_img[0]]
    title = ['Hazy Image', 'Ground Truth', 'Dehazed Image']
    
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
        
    plt.show()

# Network Function

In [59]:
def LCAnet():
    
    inputs = tf.keras.Input(shape = [412,548, 3])     # height, width of input image changed because of error in output
    conv = Conv2D(filters = 50, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu',)(inputs)
    poolLayer=AveragePooling2D(pool_size=(2,2))(conv)
    conv1 = Conv2D(filters = 50, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu')(poolLayer)
    poolLayer=AveragePooling2D(pool_size=(2,2))(conv1)  
    #flat=Flatten()(poolLayer)
    dens1=Dense(10,activation='relu')(poolLayer)
    dens2=Dense(10,activation='relu')(dens1)
    deconv1=Conv2DTranspose(50,kernel_size=(3,3),padding='same',activation='relu')(dens2)
    upsamp1=UpSampling2D(size=(2,2))(deconv1)
    deconv2=Conv2DTranspose(50,kernel_size=(3,3),padding='same',activation='relu')(upsamp1)
    upsamp2=UpSampling2D(size=(2,2))(deconv2)
    deconv3=Conv2DTranspose(3,kernel_size=(3,3),padding='same',activation='linear')(upsamp2)
    output = deconv3
    
    return Model(inputs = inputs, outputs = output)

In [60]:
model=LCAnet()
model.build([412,548,3])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 412, 548, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 412, 548, 50)      1400      
_________________________________________________________________
average_pooling2d_4 (Average (None, 206, 274, 50)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 206, 274, 50)      22550     
_________________________________________________________________
average_pooling2d_5 (Average (None, 103, 137, 50)      0         
_________________________________________________________________
dense_4 (Dense)              (None, 103, 137, 10)      510       
_________________________________________________________________
dense_5 (Dense)              (None, 103, 137, 10)      110 

In [61]:
# Hyperparameters
epochs = 10
batch_size = 8
k_init = tf.keras.initializers.random_normal(stddev=0.008, seed = 101)      
regularizer = tf.keras.regularizers.L2(1e-4)
b_init = tf.constant_initializer()

#train_data, val_data = data_path(orig_img_path = './drive/MyDrive/reside/archive/clear_images', hazy_img_path = './drive/MyDrive/reside/archive/haze')
train_data, val_data = data_path(orig_img_path = './drive/MyDrive/dataset/clear_images', hazy_img_path = './drive/MyDrive/dataset/haze')
train, val = dataloader(train_data, val_data, batch_size)

optimizer = Adam(learning_rate = 1e-4)
net = LCAnet()
#net= tf.keras.models.load_model('/content/drive/MyDrive/nets/lca',compile=False)
train_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "train loss")
val_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "val loss")

In [62]:
def train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer):
    
    for epoch in range(epochs):
        
        print("\nStart of epoch %d" % (epoch,), end=' ')
        start_time_epoch = time.time()
        start_time_step = time.time()
        
        # training loop
        
        for step, (train_batch_haze, train_batch_orig) in enumerate(train):

            with tf.GradientTape() as tape:

                train_logits = net(train_batch_haze, training = True)
                loss = mean_squared_error(train_batch_orig, train_logits)

            grads = tape.gradient(loss, net.trainable_weights)
            optimizer.apply_gradients(zip(grads, net.trainable_weights))

            train_loss_tracker.update_state(train_batch_orig, train_logits)
            if step == 0:
                print('[', end='')
            if step % 64 == 0:
                print('=', end='')
        
        print(']', end='')
        print('  -  ', end='')
        print('Training Loss: %.4f' % (train_loss_tracker.result()), end='')
        
        # validation loop
        
        for step, (val_batch_haze, val_batch_orig) in enumerate(val):
            val_logits = net(val_batch_haze, training = False)
            val_loss_tracker.update_state(val_batch_orig, val_logits)
            
            if step % 32 ==0:
                display_img(net, val_batch_haze, val_batch_orig)
        
        print('  -  ', end='')
        print('Validation Loss: %.4f' % (val_loss_tracker.result()), end='')
        print('  -  ', end=' ')
        print("Time taken: %.2fs" % (time.time() - start_time_epoch))
        
        net.save('trained_model')           # save the model(variables, weights, etc)
        train_loss_tracker.reset_states()
        val_loss_tracker.reset_states()

In [68]:
%%time
train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer)


Start of epoch 0 [=

KeyboardInterrupt: ignored

In [57]:
net.save('./drive/MyDrive/nets/lca')
model=net
model.build([512,512,3])
model.summary()

INFO:tensorflow:Assets written to: ./drive/MyDrive/nets/lca/assets
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 512, 512, 50)      1400      
_________________________________________________________________
average_pooling2d (AveragePo (None, 256, 256, 50)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 50)      22550     
_________________________________________________________________
average_pooling2d_1 (Average (None, 128, 128, 50)      0         
_________________________________________________________________
dense (Dense)                (None, 128, 128, 10)      510       
____________________________________________________________

In [21]:
import cv2
def test_model(img_path, model):

    """ 
    path : test image folder path
    model : model instantiation
    
    DRIVE SHOULD BE MOUNTED
    
    """
    
    i=0


    #plt.figure(figsize = (15,15))
    test_img = glob.glob(img_path + '/*.jpg')
    print(test_img)
    test_img_slice = tf.data.Dataset.from_tensor_slices([img for img in test_img]).map(lambda x: load_image(x))
    test = tf.data.Dataset.zip((test_img_slice)).batch(45)
    
    directory = '/content/drive/MyDrive/Test'
    os.chdir(directory)
    for img in test:
        print(img.shape)
        #pred = model(img)
        #plt.imshow(pred)
        #filename = 'Test-' + str(i) + '.jpg'
        #cv2.imwrite(filename, pred) 
        i+=1
    os.chdir('/content')

In [32]:
#test_net = tf.keras.models.load_model('trained_model', compile = False)
new_model = tf.keras.models.load_model('/content/drive/MyDrive/nets/lca',compile=False)

#test_model('/content/drive/MyDrive/ohaze/hazy',new_model)

In [96]:
import cv2
import matplotlib.pyplot as plt

def evaluate_gen(test_img_path,net):
    
    #test_img = glob.glob(test_img_path +'/*.jpg')
    test_img=glob.glob('/content/drive/MyDrive/ohaze/hazy/*.jpg')
    #random.shuffle(test_img)
    i=1;
    for img in test_img:
        
        img = tf.io.read_file(img)
        img = tf.io.decode_jpeg(img, channels = 3)
        
        img = tf.image.resize(img, size = (412,548), antialias = True)
        
        img = img / 255.0
        print(img.shape)
        img = tf.expand_dims(img, axis = 0)      #transform input image from 3D to 4D ###
        
        dehaze = net(img)
        
        #plt.figure(figsize = (80, 80))
        
        #display_list = [img[0], dehaze[0]]       #make the first dimension zero
        im=dehaze[0]
        directory = '/content/drive/MyDrive/Test'
        os.chdir(directory)
        filename = str(i) + '_outdoor_gen.jpg'
        #print(filename)
        #cv2.imwrite(filename,im) 
        #plt.imsave(filename,im)
        tf.keras.preprocessing.image.save_img(
    filename, im)

        os.chdir('/content')
        i+=1;

In [97]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/nets/lca/trained_model',compile=False)

evaluate_gen('/content/drive/MyDrive/ohaze/hazy/',new_model)

(412, 548, 3)
1_outdoor_gen.jpg
(412, 548, 3)
2_outdoor_gen.jpg
(412, 548, 3)
3_outdoor_gen.jpg
(412, 548, 3)
4_outdoor_gen.jpg
(412, 548, 3)
5_outdoor_gen.jpg
(412, 548, 3)
6_outdoor_gen.jpg
(412, 548, 3)
7_outdoor_gen.jpg
(412, 548, 3)
8_outdoor_gen.jpg
(412, 548, 3)
9_outdoor_gen.jpg
(412, 548, 3)
10_outdoor_gen.jpg
(412, 548, 3)
11_outdoor_gen.jpg
(412, 548, 3)
12_outdoor_gen.jpg
(412, 548, 3)
13_outdoor_gen.jpg
(412, 548, 3)
14_outdoor_gen.jpg
(412, 548, 3)
15_outdoor_gen.jpg
(412, 548, 3)
16_outdoor_gen.jpg
(412, 548, 3)
17_outdoor_gen.jpg
(412, 548, 3)
18_outdoor_gen.jpg
(412, 548, 3)
19_outdoor_gen.jpg
(412, 548, 3)
20_outdoor_gen.jpg
(412, 548, 3)
21_outdoor_gen.jpg
(412, 548, 3)
22_outdoor_gen.jpg
(412, 548, 3)
23_outdoor_gen.jpg
(412, 548, 3)
24_outdoor_gen.jpg
(412, 548, 3)
25_outdoor_gen.jpg
(412, 548, 3)
26_outdoor_gen.jpg
(412, 548, 3)
27_outdoor_gen.jpg
(412, 548, 3)
28_outdoor_gen.jpg
(412, 548, 3)
29_outdoor_gen.jpg
(412, 548, 3)
30_outdoor_gen.jpg
(412, 548, 3)
31_ou

In [91]:
ls

drive/  sample_data/  trained_model/


In [65]:
ls ./trained_model

assets/  saved_model.pb  variables/


In [67]:
cp -r trained_model/ /content/drive/MyDrive/nets/lca/trained_model